In [15]:
%env PATH=/appl/gams/47.6.0:$PATH
%env LD_LIBRARY_PATH=/appl/gams/47.6.0:$LD_LIBRARY_PATH

env: PATH=/appl/gams/47.6.0:$PATH
env: LD_LIBRARY_PATH=/appl/gams/47.6.0:$LD_LIBRARY_PATH


In [16]:

import Functions_analysis as FA
import os

YEAR = '2050'

Nordics = ['DENMARK', 'NORWAY', 'SWEDEN', 'FINLAND']
EU_North = ['DENMARK', 'NORWAY', 'SWEDEN', 'FINLAND', 'ESTONIA', 'LATVIA', 'LITHUANIA']
EU_West = ['FRANCE', 'GERMANY', 'NETHERLANDS', 'UNITED-KINGDOM', 'BELGIUM', 'LUXEMBOURG', 'AUSTRIA', 'SWITZERLAND', 'IRELAND']
EU_East = ['POLAND', 'CZECH_REPUBLIC', 'SLOVAKIA', 'ROMANIA', 'BULGARIA', 'HUNGARY']
EU_South = ['ITALY', 'SPAIN', 'PORTUGAL', 'SLOVENIA', 'CROATIA', 'ALBANIA', 'MALTA', 'CYPRUS', 'BOSNIA_AND_HERZEGOVINA', 'MONTENEGRO', 'NORTH_MACEDONIA', 'SERBIA', 'GREECE']

Input_data_path = f"../DATA/Input_data_EU_{YEAR}.gdx"
MainResults_path = f"../DATA/MainResults_EU_{YEAR}.gdx"
MonteCarlo_path = f"../DATA/MonteCarlo_EU_{YEAR}.gdx"

MainResults_path_NoH2 = f"../DATA/MainResults_EU_{YEAR}_NoH2.gdx"
MainResults_path_NoH2Import = f"../DATA/MainResults_EU_{YEAR}_NoH2Import.gdx"

MonteCarlo_path_NoH2 = f"../DATA/MonteCarlo_EU_{YEAR}_NoH2.gdx"
MonteCarlo_path_NoH2Import = f"../DATA/MonteCarlo_EU_{YEAR}_NoH2Import.gdx"

In [17]:
def Hydrogen_Analysis(MainResults_path, MonteCarlo_path, YEAR, Countries_from) :
    # Import results of the Main Result file
    df_PRO_BASE, df_CAP_BASE, df_XH2_CAP_BASE, df_XH2_FLOW_BASE = FA.Import_MainResults(MainResults_path)
    
    # Export all hydrogen results
    df_H2_CAP_BASE, df_H2_CAP_GREEN_BASE, df_H2_CAP_GREEN_tot_BASE, df_H2_CAP_BLUE_BASE, df_H2_CAP_BLUE_tot_BASE, df_H2_CAP_STO_BASE, df_H2_CAP_STO_tot_BASE = FA.H2_CAP(df_CAP_BASE, Countries_from, YEAR)
    df_H2_PRO_BASE, df_H2_PRO_GREEN_BASE, df_H2_PRO_GREEN_tot_BASE, df_H2_PRO_BLUE_BASE, df_H2_PRO_BLUE_tot_BASE, df_H2_PRO_STO_BASE, df_H2_PRO_STO_tot_BASE = FA.H2_PRO(df_PRO_BASE, df_CAP_BASE, Countries_from, YEAR)
    dict_df_XH2_CAP_TO_BASE = FA.XH2(df_XH2_CAP_BASE, Countries_from, YEAR, 'dict')
    dict_df_XH2_FLOW_TO_BASE = FA.XH2(df_XH2_FLOW_BASE, Countries_from, YEAR, 'dict')
        
    # Import resuls of the MonteCarlo file
    df_PRO_scen, df_CAP_scen, df_XH2_CAP_scen, df_XH2_FLOW_scen, scen = FA.Import_MonteCarlo(MonteCarlo_path)
    
    # Export all hydrogen results
    df_H2_CAP_scen, df_H2_CAP_GREEN_scen, df_H2_CAP_BLUE_scen, df_H2_CAP_STO_scen = FA.H2_CAP_scen(df_CAP_scen, scen, Countries_from, YEAR)
    df_H2_PRO_scen, df_H2_PRO_GREEN_scen, df_H2_PRO_BLUE_scen, df_H2_PRO_STO_scen = FA.H2_PRO_scen(df_PRO_scen, df_CAP_scen, scen, Countries_from, YEAR)
    df_XH2_CAP_tot_scen, dict_df_XH2_CAP_TO_scen = FA.XH2_scen(df_XH2_CAP_scen, scen, Countries_from, YEAR, 'dict')
    df_XH2_FLOW_tot_scen, dict_df_XH2_FLOW_TO_scen = FA.XH2_scen(df_XH2_FLOW_scen, scen, Countries_from, YEAR, 'dict')
    
    ECDF_Hist_PRO = FA.ECDF_Hist_PRO(df_H2_PRO_GREEN_scen, df_H2_PRO_BLUE_scen, df_H2_PRO_STO_scen, df_H2_PRO_GREEN_tot_BASE, df_H2_PRO_BLUE_tot_BASE, df_H2_PRO_STO_tot_BASE, Countries_from, YEAR)
    ECDF_Hist_CAP = FA.ECDF_Hist_CAP(df_H2_CAP_GREEN_scen, df_H2_CAP_BLUE_scen, df_H2_CAP_GREEN_tot_BASE, df_H2_CAP_BLUE_tot_BASE, Countries_from, YEAR)
    Violin_PRO = FA.Violin_PRO(df_H2_PRO_GREEN_scen, df_H2_PRO_BLUE_scen, df_H2_PRO_STO_scen, df_H2_PRO_GREEN_tot_BASE, df_H2_PRO_BLUE_tot_BASE, df_H2_PRO_STO_tot_BASE, Countries_from, YEAR)
    Violin_CAP = FA.Violin_CAP(df_H2_CAP_GREEN_scen, df_H2_CAP_BLUE_scen, df_H2_CAP_STO_scen, df_H2_CAP_GREEN_tot_BASE, df_H2_CAP_BLUE_tot_BASE, df_H2_CAP_STO_tot_BASE, Countries_from, YEAR)
    BoxPlot_CAP = FA.BoxPlot_Transmission(dict_df_XH2_CAP_TO_BASE, dict_df_XH2_CAP_TO_scen, Countries_from, YEAR, "Capacity")
    BoxPlot_FLOW = FA.BoxPlot_Transmission(dict_df_XH2_FLOW_TO_BASE, dict_df_XH2_FLOW_TO_scen, Countries_from, YEAR, "Flow")

In [18]:
def Hydrogen_Setups_Analysis(MainResults_path, MainResults_path_NoH2, MainResults_path_NoH2Import, MonteCarlo_path, MonteCarlo_path_NoH2, MonteCarlo_path_NoH2Import, YEAR, Countries_from) :
    # Import results of the Main Result file
    df_PRO_BASE, df_CAP_BASE, df_XH2_CAP_BASE, df_XH2_FLOW_BASE = FA.Import_MainResults(MainResults_path)
    df_PRO_BASE_NoH2, df_CAP_BASE_NoH2, df_XH2_CAP_BASE_NoH2, df_XH2_FLOW_BASE_NoH2 = FA.Import_MainResults(MainResults_path_NoH2)
    df_PRO_BASE_NoH2Import, df_CAP_BASE_NoH2Import, df_XH2_CAP_BASE_NoH2Import, df_XH2_FLOW_BASE_NoH2Import = FA.Import_MainResults(MainResults_path_NoH2Import)
    
    # Export all hydrogen results
    # Base case
    df_H2_CAP_BASE, df_H2_CAP_GREEN_BASE, df_H2_CAP_GREEN_tot_BASE, df_H2_CAP_BLUE_BASE, df_H2_CAP_BLUE_tot_BASE, df_H2_CAP_STO_BASE, df_H2_CAP_STO_tot_BASE = FA.H2_CAP(df_CAP_BASE, Countries_from, YEAR)
    df_H2_PRO_BASE, df_H2_PRO_GREEN_BASE, df_H2_PRO_GREEN_tot_BASE, df_H2_PRO_BLUE_BASE, df_H2_PRO_BLUE_tot_BASE, df_H2_PRO_STO_BASE, df_H2_PRO_STO_tot_BASE = FA.H2_PRO(df_PRO_BASE, df_CAP_BASE, Countries_from, YEAR)
    XH2_CAP_TO_BASE = FA.XH2(df_XH2_CAP_BASE, Countries_from, YEAR, 'total')
    XH2_FLOW_TO_BASE = FA.XH2(df_XH2_FLOW_BASE, Countries_from, YEAR, 'total')
    # No H2 case
    df_H2_CAP_NoH2, df_H2_CAP_GREEN_NoH2, df_H2_CAP_GREEN_tot_NoH2, df_H2_CAP_BLUE_NoH2, df_H2_CAP_BLUE_tot_NoH2, df_H2_CAP_STO_NoH2, df_H2_CAP_STO_tot_NoH2 = FA.H2_CAP(df_CAP_BASE_NoH2, Countries_from, YEAR)
    df_H2_PRO_NoH2, df_H2_PRO_GREEN_NoH2, df_H2_PRO_GREEN_tot_NoH2, df_H2_PRO_BLUE_NoH2, df_H2_PRO_BLUE_tot_NoH2, df_H2_PRO_STO_NoH2, df_H2_PRO_STO_tot_NoH2 = FA.H2_PRO(df_PRO_BASE_NoH2, df_CAP_BASE_NoH2, Countries_from, YEAR)
    XH2_CAP_TO_NoH2 = FA.XH2(df_XH2_CAP_BASE_NoH2, Countries_from, YEAR, 'total')
    XH2_FLOW_TO_NoH2 = FA.XH2(df_XH2_FLOW_BASE_NoH2, Countries_from, YEAR, 'total')
    # No H2 Import case
    df_H2_CAP_NoH2Import, df_H2_CAP_GREEN_NoH2Import, df_H2_CAP_GREEN_tot_NoH2Import, df_H2_CAP_BLUE_NoH2Import, df_H2_CAP_BLUE_tot_NoH2Import, df_H2_CAP_STO_NoH2Import, df_H2_CAP_STO_tot_NoH2Import = FA.H2_CAP(df_CAP_BASE_NoH2Import, Countries_from, YEAR)
    df_H2_PRO_NoH2Import, df_H2_PRO_GREEN_NoH2Import, df_H2_PRO_GREEN_tot_NoH2Import, df_H2_PRO_BLUE_NoH2Import, df_H2_PRO_BLUE_tot_NoH2Import, df_H2_PRO_STO_NoH2Import, df_H2_PRO_STO_tot_NoH2Import = FA.H2_PRO(df_PRO_BASE_NoH2Import, df_CAP_BASE_NoH2Import, Countries_from, YEAR)
    XH2_CAP_TO_NoH2Import = FA.XH2(df_XH2_CAP_BASE_NoH2Import, Countries_from, YEAR, 'total')
    XH2_FLOW_TO_NoH2Import = FA.XH2(df_XH2_FLOW_BASE_NoH2Import, Countries_from, YEAR, 'total')
    
    # Import resuls of the MonteCarlo file
    df_PRO_BASE_scen, df_CAP_BASE_scen, df_XH2_CAP_BASE_scen, df_XH2_FLOW_BASE_scen, scen = FA.Import_MonteCarlo(MonteCarlo_path)
    df_PRO_NoH2_scen, df_CAP_NoH2_scen, df_XH2_CAP_NoH2_scen, df_XH2_FLOW_NoH2_scen, scen = FA.Import_MonteCarlo(MonteCarlo_path_NoH2)
    df_PRO_NoH2Import_scen, df_CAP_NoH2Import_scen, df_XH2_CAP_NoH2Import_scen, df_XH2_FLOW_NoH2Import_scen, scen = FA.Import_MonteCarlo(MonteCarlo_path_NoH2Import)
    
    # Export all hydrogen results
    # Base case
    df_H2_CAP_BASE_scen, df_H2_CAP_GREEN_BASE_scen, df_H2_CAP_BLUE_BASE_scen, df_H2_CAP_STO_BASE_scen = FA.H2_CAP_scen(df_CAP_BASE_scen, scen, Countries_from, YEAR)
    df_H2_PRO_BASE_scen, df_H2_PRO_GREEN_BASE_scen, df_H2_PRO_BLUE_BASE_scen, df_H2_PRO_STO_BASE_scen = FA.H2_PRO_scen(df_PRO_BASE_scen, df_CAP_BASE_scen, scen, Countries_from, YEAR)
    df_XH2_CAP_tot_BASE_scen = FA.XH2_scen(df_XH2_CAP_BASE_scen, scen, Countries_from, YEAR, 'total')
    df_XH2_FLOW_tot_BASE_scen = FA.XH2_scen(df_XH2_FLOW_BASE_scen, scen, Countries_from, YEAR, 'total')
    # No H2 case
    df_H2_CAP_NoH2_scen, df_H2_CAP_GREEN_NoH2_scen, df_H2_CAP_BLUE_NoH2_scen, df_H2_CAP_STO_NoH2_scen = FA.H2_CAP_scen(df_CAP_NoH2_scen, scen, Countries_from, YEAR)
    df_H2_PRO_NoH2_scen, df_H2_PRO_GREEN_NoH2_scen, df_H2_PRO_BLUE_NoH2_scen, df_H2_PRO_STO_NoH2_scen = FA.H2_PRO_scen(df_PRO_NoH2_scen, df_CAP_NoH2_scen, scen, Countries_from, YEAR)
    df_XH2_CAP_tot_NoH2_scen = FA.XH2_scen(df_XH2_CAP_NoH2_scen, scen, Countries_from, YEAR, 'total')
    df_XH2_FLOW_tot_NoH2_scen = FA.XH2_scen(df_XH2_FLOW_NoH2_scen, scen, Countries_from, YEAR, 'total')
    # No H2 Import case
    df_H2_CAP_NoH2Import_scen, df_H2_CAP_GREEN_NoH2Import_scen, df_H2_CAP_BLUE_NoH2Import_scen, df_H2_CAP_STO_NoH2Import_scen = FA.H2_CAP_scen(df_CAP_NoH2Import_scen, scen, Countries_from, YEAR)
    df_H2_PRO_NoH2Import_scen, df_H2_PRO_GREEN_NoH2Import_scen, df_H2_PRO_BLUE_NoH2Import_scen, df_H2_PRO_STO_NoH2Import_scen = FA.H2_PRO_scen(df_PRO_NoH2Import_scen, df_CAP_NoH2Import_scen, scen, Countries_from, YEAR)
    df_XH2_CAP_tot_NoH2Import_scen = FA.XH2_scen(df_XH2_CAP_NoH2Import_scen, scen, Countries_from, YEAR, 'total')
    df_XH2_FLOW_tot_NoH2Import_scen = FA.XH2_scen(df_XH2_FLOW_NoH2Import_scen, scen, Countries_from, YEAR, 'total')
    
    FA.Violin_Setups(df_H2_CAP_GREEN_BASE_scen, df_H2_CAP_GREEN_NoH2_scen, df_H2_CAP_GREEN_NoH2Import_scen, df_H2_CAP_GREEN_tot_BASE, df_H2_CAP_GREEN_tot_NoH2, df_H2_CAP_GREEN_tot_NoH2Import, Countries_from, YEAR, "Green Capacity")
    FA.Violin_Setups(df_H2_CAP_BLUE_BASE_scen, df_H2_CAP_BLUE_NoH2_scen, df_H2_CAP_BLUE_NoH2Import_scen, df_H2_CAP_BLUE_tot_BASE, df_H2_CAP_BLUE_tot_NoH2, df_H2_CAP_BLUE_tot_NoH2Import, Countries_from, YEAR, "Blue Capacity")
    FA.Violin_Setups(df_H2_PRO_GREEN_BASE_scen, df_H2_PRO_GREEN_NoH2_scen, df_H2_PRO_GREEN_NoH2Import_scen, df_H2_PRO_GREEN_tot_BASE, df_H2_PRO_GREEN_tot_NoH2, df_H2_PRO_GREEN_tot_NoH2Import, Countries_from, YEAR, "Green Production")
    FA.Violin_Setups(df_H2_PRO_BLUE_BASE_scen, df_H2_PRO_BLUE_NoH2_scen, df_H2_PRO_BLUE_NoH2Import_scen, df_H2_PRO_BLUE_tot_BASE, df_H2_PRO_BLUE_tot_NoH2, df_H2_PRO_BLUE_tot_NoH2Import, Countries_from, YEAR, "Blue Production")
    FA.Violin_Setups(df_H2_PRO_STO_BASE_scen, df_H2_PRO_STO_NoH2_scen, df_H2_PRO_STO_NoH2Import_scen, df_H2_PRO_STO_tot_BASE, df_H2_PRO_STO_tot_NoH2, df_H2_PRO_STO_tot_NoH2Import, Countries_from, YEAR, "Storage")
    FA.Violin_Setups(df_XH2_CAP_tot_BASE_scen, df_XH2_CAP_tot_NoH2_scen, df_XH2_CAP_tot_NoH2Import_scen, XH2_CAP_TO_BASE, XH2_CAP_TO_NoH2, XH2_CAP_TO_NoH2Import, Countries_from, YEAR, "Transmission Capacity")
    FA.Violin_Setups(df_XH2_FLOW_tot_BASE_scen, df_XH2_FLOW_tot_NoH2_scen, df_XH2_FLOW_tot_NoH2Import_scen, XH2_FLOW_TO_BASE, XH2_FLOW_TO_NoH2, XH2_FLOW_TO_NoH2Import, Countries_from, YEAR, "Transmission Flow")
    

In [19]:
def CCS_Analysis(MainResults_path, YEAR) :
    # Import results of the Main Result file
    result = FA.Import_CCS(MainResults_path,YEAR)
    print(f"The total captured CO2 for {YEAR} is {result} kton")
    return result

In [20]:
def RE_Analysis(Input_data_path, MonteCarlo_path, Countries, YEAR) :
    df_SUBTECH = FA.Import_Data(Input_data_path)
    df_RE_SUBTECH = FA.LIM_RE_CAP_scen(df_SUBTECH)
    df_PRO, df_CAP, df_XH2_CAP, df_XH2_FLOW, scen = FA.Import_MonteCarlo(MonteCarlo_path)
    df_RE_CAP = FA.RE_CAP_scen(df_CAP, YEAR)
    
    FA.RE_CAP(df_RE_CAP, Countries, YEAR)
    FA.COMP_RE_LIM(df_RE_SUBTECH, df_RE_CAP, Countries, YEAR)
    